In [12]:
from gensim.models.keyedvectors import FastTextKeyedVectors

import pandas as pd
from os import listdir
from os.path import isfile, join

import progressbar
import time


In [2]:
fasttext = FastTextKeyedVectors.load("./FASTTEXT//araneum_none_fasttextcbow_300_5_2018.model")

In [10]:
df = pd.read_csv("./sub/proessed_subs/Friends - 5x19.srt.csv", sep='\t', encoding='cp1251')
df.head()

,cleaned_text,lemm_text
0,Ух ты! Выглядишь прекрасно. Что за событие?,ух ты выглядеть прекрасно что событие
1,Моника и я отмечаем нашу десятимесячную годовщ...,моника отмечать наш десятимесячный годовщина
2,"Я заказал столик в ""Жан-Жорж"".",я заказать столик жанжорж
3,Как ты попал туда?,как попасть туда
4,"Сделал пару звонков, дёрнул пару рычажков...",сделать пара звонок дёрнуть пара рычажок


In [19]:
food_word = ['кушать','покушать','еда','пища','блюдо','суп','пир','питье',
             'выпивка' ,'закуска','жратва','угощение','лакомство','ужин','завтрак','напиток',
            'щи','каша','борщ','похлебка','свинина','говядина','пирог','квашеный','котлета','мясо','баранина','курятина',
            'овощ', 'фрукт']

def detect_food(line):
    try:
        words = line.split()
    except:
        return False, None
    for word in words:
        for food in  food_word:
            try:
                sim = fasttext.similarity(word, food)
                if(sim > 0.8):
                    print(line, ":",word,  food, sim)
                    food_stc = [line, word, food]
                    return True, food_stc
            except:
                pass
                
    return False, None

In [14]:
def get_food_dialogues(cleaned_text, lemm_text, debug = False):

    bar = progressbar.ProgressBar(maxval=len(lemm_text),
                                  widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    lines = 0

    dialogues_phrases = []
    current_phrase = []
    similarity_dict = []
    add_response_next_iter = False

    ITERATION = 0

    stop = 0

    for line, orig_line in zip(lemm_text, cleaned_text):
        #print(line, orig_line, ITERATION)
        if(add_response_next_iter):
            if(debug): print("SECOND:", orig_line, ITERATION)
            current_phrase.append(orig_line)
            dialogues_phrases.append(current_phrase)
            assert (len(current_phrase) == 2)
            add_response_next_iter = False
            current_phrase = []
            
        
        add_response_next_iter, similarity = detect_food(line)

        if(similarity):
            if(debug):
                print(similarity, ITERATION)
                print(line, orig_line, ITERATION)
            similarity_dict.append(similarity)

        if(add_response_next_iter):
            if(debug): print("FIRST:", orig_line, ITERATION)
            current_phrase.append(orig_line)
            assert (len(current_phrase) == 1)

        ITERATION += 1

        """
        stop += 1
        if stop > 100:
            break"""

        lines += 1
        bar.update(lines)
        time.sleep(0.1)
        
    return dialogues_phrases

In [20]:
get_food_dialogues(df['cleaned_text'], df['lemm_text'], debug = True)

сашка скоро завтракать собираться : завтракать кушать 0.801914
['сашка скоро завтракать собираться', 'завтракать', 'кушать'] 43
сашка скоро завтракать собираться -Сашка, ты скоро? Завтракать собираешься? 43
FIRST: -Сашка, ты скоро? Завтракать собираешься? 43
SECOND: Лина ждёт тебя голодная. -Спасибо. 44


смеяться ладно завтракать : завтракать кушать 0.801914
['смеяться ладно завтракать', 'завтракать', 'кушать'] 52
смеяться ладно завтракать СМЕЁТСЯ Ладно, завтракайте. 52
FIRST: СМЕЁТСЯ Ладно, завтракайте. 52
SECOND: Приятного аппетита. 53


KeyboardInterrupt: 

In [28]:
subs_list = listdir("./sub/proessed_subs/")[19:]
subs_list

['1-2 серии (AVS).srt.csv',
 '10 серия (AMZN.WEBRip.DDP5.1.x264).srt.csv',
 '10.5.minut.ticiny.vozvrachenie.2018.WEB-DLRip.Files-x.srt.csv',
 '10.Chernov.(2018).WEB-DLRip.Files-x.srt.csv',
 '11.5.minut.ticiny.vozvrachenie.2018.WEB-DLRip.Files-x.srt.csv',
 '11.Chernov.(2018).WEB-DLRip.Files-x.srt.csv',
 '12.5.minut.ticiny.vozvrachenie.2018.WEB-DLRip.Files-x.srt.csv',
 '12.Chernov.(2018).WEB-DLRip.Files-x.srt.csv',
 '13.5.minut.ticiny.vozvrachenie.2018.WEB-DLRip.Files-x.srt.csv',
 '14.5.minut.ticiny.vozvrachenie.2018.WEB-DLRip.Files-x.srt.csv',
 '15.5.minut.ticiny.vozvrachenie.2018.WEB-DLRip.Files-x.srt.csv',
 '16.5.minut.ticiny.vozvrachenie.2018.WEB-DLRip.Files-x.srt.csv',
 '17.5.minut.ticiny.vozvrachenie.2018.WEB-DLRip.Files-x.srt.csv',
 '18.5.minut.ticiny.vozvrachenie.2018.WEB-DLRip.Files-x.srt.csv',
 '19.5.minut.ticiny.vozvrachenie.2018.WEB-DLRip.Files-x.srt.csv',
 '2 серия (AMZN.WEB-DL.DDP5.1.H.264).srt.csv',
 '20.5.minut.ticiny.vozvrachenie.2018.WEB-DLRip.Files-x.srt.csv',
 '21.5.m

In [33]:
subs = listdir("./sub/proessed_subs/")[:19]

In [34]:
all_subs_dialogues = []
for sub_df in subs:
    path_to_df = join("./sub/proessed_subs/",sub_df)
    print("NOW PROCESSING",path_to_df)
    try:
        df = pd.read_csv(path_to_df, sep='\t', encoding='cp1251')
        film_phrases = get_food_dialogues(df['cleaned_text'], df['lemm_text'], debug = True)
        all_subs_dialogues.extend(film_phrases)
    except:
        print("FILE_READING_FAILED")
        pass
    

NOW PROCESSING ./sub/proessed_subs/01.5.minut.ticiny.vozvrachenie.2018.WEB-DLRip.Files-x.srt.csv


сашка скоро завтракать собираться : завтракать кушать 0.801914
['сашка скоро завтракать собираться', 'завтракать', 'кушать'] 43
сашка скоро завтракать собираться -Сашка, ты скоро? Завтракать собираешься? 43
FIRST: -Сашка, ты скоро? Завтракать собираешься? 43
SECOND: Лина ждёт тебя голодная. -Спасибо. 44


смеяться ладно завтракать : завтракать кушать 0.801914
['смеяться ладно завтракать', 'завтракать', 'кушать'] 52
смеяться ладно завтракать СМЕЁТСЯ Ладно, завтракайте. 52
FIRST: СМЕЁТСЯ Ладно, завтракайте. 52
SECOND: Приятного аппетита. 53


чай недавно пить : пить кушать 0.817057
['чай недавно пить', 'пить', 'кушать'] 374
чай недавно пить -А я чай недавно пила. 374
FIRST: -А я чай недавно пила. 374
SECOND: -Правда. Я воду из фильтра брала. 375


борщ сваривать хватить издеваться : борщ борщ 1.0
['борщ сваривать хватить издеваться', 'борщ', 'борщ'] 439
борщ сваривать хватить издеваться -А борщ-то не сварен. -Хватит издеваться. 439
FIRST: -А борщ-то не сварен. -Хватит издеваться. 439
SECOND: Да, борщ-то и не сварен. 440
борщ сваривать : борщ борщ 1.0
['борщ сваривать', 'борщ', 'борщ'] 440
борщ сваривать Да, борщ-то и не сварен. 440
FIRST: Да, борщ-то и не сварен. 440
SECOND: Сегодня котлеты на ужин. 441
сегодня котлета ужин : котлета котлета 1.0
['сегодня котлета ужин', 'котлета', 'котлета'] 441
сегодня котлета ужин Сегодня котлеты на ужин. 441
FIRST: Сегодня котлеты на ужин. 441
SECOND: Хочешь поболтать - поехали со мной. -Поехали. 442


...

NOW PROCESSING ./sub/proessed_subs/09.Chernov.(2018).WEB-DLRip.Files-x.srt.csv


хотеть выпивать пить : пить кушать 0.817057
['хотеть выпивать пить', 'пить', 'кушать'] 408
хотеть выпивать пить Я хочу выпить. Пейте. 408
FIRST: Я хочу выпить. Пейте. 408
SECOND: А вы так со всеми женщинами галантны или только со мной? 409


наливать вино хотеть ... хотеть пить вино : пить кушать 0.817057
['наливать вино хотеть ... хотеть пить вино', 'пить', 'кушать'] 413
наливать вино хотеть ... хотеть пить вино Налейте мне вина. Я хочу... я хочу пить вино. 413
FIRST: Налейте мне вина. Я хочу... я хочу пить вино. 413
SECOND: Вы знаете... 414


давать чай приносить доктор говорить нужно пить чай : пить кушать 0.817057
['давать чай приносить доктор говорить нужно пить чай', 'пить', 'кушать'] 503
давать чай приносить доктор говорить нужно пить чай Давай я тебе чай принесу. Доктор говорит, тебе нужно пить чай. 503
FIRST: Давай я тебе чай принесу. Доктор говорит, тебе нужно пить чай. 503
SECOND: -Давай. 504


NOW PROCESSING ./sub/proessed_subs/1 серия (AMZN.WEB-DL.DDP5.1.H.264).srt.csv
FILE_READING_FAILED


In [44]:
len(all_subs_dialogues)

93

In [36]:
def save_dialogues(dialogues, line_number):
    fist_phrase = [phrase[0] for phrase in dialogues]
    reply = [phrase[1] for phrase in dialogues]
    dialogues_df = pd.DataFrame(
    {'fist_phrase': fist_phrase,
     'reply': reply,
    })
    df_name = "C:\HSE_learning\Python\collect_dataset\sub\dialogues_df_" + str(line_number) +".csv"
    try:
        dialogues_df.to_csv(df_name, sep='\t', encoding='utf-8', index=False)
    except:
        print('SAVE_FAILED')
        return
    print("saved", df_name)

In [37]:
save_dialogues(all_subs_dialogues, "all_rest")

saved C:\HSE_learning\Python\collect_dataset\sub\dialogues_df_all_rest.csv


In [75]:
big_df = pd.read_csv("C:\HSE_learning\Python\collect_dataset\sub\dialogues_df_all.csv", sep='\t', encoding='utf-8')

In [77]:
small_df = pd.read_csv("C:/HSE_learning/Python/collect_dataset/sub/dialogues_df_all_rest.csv", sep='\t', encoding='utf-8')

In [70]:
big_df.head(50)

,fist_phrase,reply
0,"-Кать,"" оставайтесь с нами на ужин.","Андрея всё равно нет. Поужинаете у нас."",,,,,,"
1,Андрея всё равно нет. Поужинаете у нас.\tПодож...,NaN
2,"-Не понял. -Катя,"" оставайтесь с нами на ужин.","А дайте поцелую вашу ручку."",,,,,,"
3,"Ты что,"" едой не хочешь делиться? -Я тобой не ...","-Ты что"", ревнуешь? Мне так нравится, когда ты..."
4,"-Ничего,"" сейчас у них обед.","Обходить нам"", что ли?,,,,,"
5,А пока пообедаем в вашем ресторане за ваш счёт...,NaN
6,-А вызывайте. А мы пока пообедаем. Официант!\t...,NaN
7,"#NAME?,"" конечно. Только сначала пообедаем.","-Ну"", что у нас там сегодня?,,,,,"
8,-Это ошибка какая-то. Мы пришли сюда пообедать...,NaN
9,-Вообще-то я тебе ужин приготовила.\tВино купи...,NaN


In [55]:
len(big_df)

650

In [46]:
fist_phrase = [phrase[0] for phrase in all_subs_dialogues]
reply = [phrase[1] for phrase in all_subs_dialogues]
len(fist_phrase)

93

In [73]:
final_df = pd.concat([big_df, small_df], ignore_index=True, axis = 0)
final_df["fist_phrase"][1], final_df["reply"][1]

('Андрея всё равно нет. Поужинаете у нас.\tПодождёте здесь. -Спасибо, конечно, за приглашение,,,,,',
 nan)

In [61]:
len(final_df)

743

In [96]:
final_df.to_csv("C:\HSE_learning\Python\collect_dataset\sub\dialogues_df_final.csv", sep='\t', encoding='utf-8', index=False)

In [67]:
final = pd.read_csv("C:/HSE_learning/Python/collect_dataset/sub/dialogues_df_final.csv", sep='\t', encoding='utf-8')

In [78]:
final['fist_phrase'][5]

'А пока пообедаем в вашем ресторане за ваш счёт.\t-Знаете что, а вот это уже хамство. Вы хотите, чтобы я вызвала полицию?,,,,,'

In [88]:
st =0

new_first = []
new_reply = []

for first, reply in zip(final['fist_phrase'], final['reply']):
    
    #print(first,"|||", reply)
    if '\t' in first:
        fst, rpl = first.split('\t')
        #print(fst, ">>>", rpl)
        new_first.append(fst)
        new_reply.append(rpl)
        
    else:
        new_first.append(first)
        new_reply.append(reply)
    
    """
    st+=1
    if st > 10:
        break"""

In [89]:
new_first[:10]

['-Кать," оставайтесь с нами на ужин.',
 'Андрея всё равно нет. Поужинаете у нас.',
 '-Не понял. -Катя," оставайтесь с нами на ужин.',
 'Ты что," едой не хочешь делиться? -Я тобой не хочу делиться.',
 '-Ничего," сейчас у них обед.',
 'А пока пообедаем в вашем ресторане за ваш счёт.',
 '-А вызывайте. А мы пока пообедаем. Официант!',
 '#NAME?," конечно. Только сначала пообедаем.',
 '-Это ошибка какая-то. Мы пришли сюда пообедать.',
 '-Вообще-то я тебе ужин приготовила.']

In [90]:
new_reply[:10]

['Андрея всё равно нет. Поужинаете у нас.",,,,,,',
 'Подождёте здесь. -Спасибо, конечно, за приглашение,,,,,',
 'А дайте поцелую вашу ручку.",,,,,,',
 '-Ты что", ревнуешь? Мне так нравится, когда ты ревнуешь.,,,,',
 'Обходить нам", что ли?,,,,,',
 '-Знаете что, а вот это уже хамство. Вы хотите, чтобы я вызвала полицию?,,,,,',
 '-Хорошо, что всё без особых проблем прошло.,,,,,,',
 '-Ну", что у нас там сегодня?,,,,,',
 'А что, нельзя?,,,,,,',
 'Вино купила.,,,,,,,']

In [94]:
dialogues_df = pd.DataFrame(
    {'fist_phrase': new_first,
     'reply': new_reply,
    })

dialogues_df.head(5)

,fist_phrase,reply
0,"-Кать,"" оставайтесь с нами на ужин.","Андрея всё равно нет. Поужинаете у нас."",,,,,,"
1,Андрея всё равно нет. Поужинаете у нас.,"Подождёте здесь. -Спасибо, конечно, за приглаш..."
2,"-Не понял. -Катя,"" оставайтесь с нами на ужин.","А дайте поцелую вашу ручку."",,,,,,"
3,"Ты что,"" едой не хочешь делиться? -Я тобой не ...","-Ты что"", ревнуешь? Мне так нравится, когда ты..."
4,"-Ничего,"" сейчас у них обед.","Обходить нам"", что ли?,,,,,"


In [97]:
dialogues_df.to_csv("C:\HSE_learning\Python\collect_dataset\sub\dialogues_df_repaire.csv", sep='\t', encoding='utf-8', index=False)

In [99]:
repaired = pd.read_csv("C:\HSE_learning\Python\collect_dataset\sub\dialogues_df_repaire.csv", sep='\t', encoding='utf-8')

In [103]:
len(repaired)

743

In [101]:
repaired.head(150)

,fist_phrase,reply
0,"-Кать,"" оставайтесь с нами на ужин.","Андрея всё равно нет. Поужинаете у нас."",,,,,,"
1,Андрея всё равно нет. Поужинаете у нас.,"Подождёте здесь. -Спасибо, конечно, за приглаш..."
2,"-Не понял. -Катя,"" оставайтесь с нами на ужин.","А дайте поцелую вашу ручку."",,,,,,"
3,"Ты что,"" едой не хочешь делиться? -Я тобой не ...","-Ты что"", ревнуешь? Мне так нравится, когда ты..."
4,"-Ничего,"" сейчас у них обед.","Обходить нам"", что ли?,,,,,"
5,А пока пообедаем в вашем ресторане за ваш счёт.,"-Знаете что, а вот это уже хамство. Вы хотите,..."
6,-А вызывайте. А мы пока пообедаем. Официант!,"-Хорошо, что всё без особых проблем прошло.,,,,,,"
7,"#NAME?,"" конечно. Только сначала пообедаем.","-Ну"", что у нас там сегодня?,,,,,"
8,-Это ошибка какая-то. Мы пришли сюда пообедать.,"А что, нельзя?,,,,,,"
9,-Вообще-то я тебе ужин приготовила.,"Вино купила.,,,,,,,"


In [69]:
final.head(50)

,fist_phrase,reply
0,"-Кать,"" оставайтесь с нами на ужин.","Андрея всё равно нет. Поужинаете у нас."",,,,,,"
1,Андрея всё равно нет. Поужинаете у нас.\tПодож...,NaN
2,"-Не понял. -Катя,"" оставайтесь с нами на ужин.","А дайте поцелую вашу ручку."",,,,,,"
3,"Ты что,"" едой не хочешь делиться? -Я тобой не ...","-Ты что"", ревнуешь? Мне так нравится, когда ты..."
4,"-Ничего,"" сейчас у них обед.","Обходить нам"", что ли?,,,,,"
5,А пока пообедаем в вашем ресторане за ваш счёт...,NaN
6,-А вызывайте. А мы пока пообедаем. Официант!\t...,NaN
7,"#NAME?,"" конечно. Только сначала пообедаем.","-Ну"", что у нас там сегодня?,,,,,"
8,-Это ошибка какая-то. Мы пришли сюда пообедать...,NaN
9,-Вообще-то я тебе ужин приготовила.\tВино купи...,NaN


In [58]:
big_df['fist_phrase'] =  big_df['fist_phrase'].extend(fist_phrase)
big_df['reply'] =  big_df['reply'].extend(reply)

len(big_df)

AttributeError: 'Series' object has no attribute 'extend'

In [41]:
small_df.head()

,fist_phrase,reply
0,"-Сашка, ты скоро? Завтракать собираешься?",Лина ждёт тебя голодная. -Спасибо.
1,"СМЕЁТСЯ Ладно, завтракайте.",Приятного аппетита.
2,-А я чай недавно пила.,-Правда. Я воду из фильтра брала.
3,-А борщ-то не сварен. -Хватит издеваться.,"Да, борщ-то и не сварен."
4,"Да, борщ-то и не сварен.",Сегодня котлеты на ужин.
